In [14]:
from fastai.vision.all import *
import torchvision 
import numpy as np
import torch
import cv2
import os
from modules_th.video_block import *
import random
import time
from pathlib import Path
import os

In [38]:
%config Completer.use_jedi = False

In [27]:
path = Path(r'C:\Users\adria_000\Desktop\malas')
vid_paths = get_files(path, extensions='.AVI')
vid_path = random.choice(vid_paths)
for i, frame in enumerate(iterator_over_frames(vid_path)):
    print(frame.__array_interface__['data'][0])
    if i >= 3: break

2271932654256
2271932654256
2271932654256
2271932654256


In [122]:
def initialize_tensors(bs, l, size):
    fr_ts = torch.zeros(3, size, size, dtype=float)
    snip = torch.zeros(l, 3, size, size, dtype=float)
    b = torch.zeros(bs, l, 3, size, size, dtype=float)
    return fr_ts, snip, b

def iterate_with_vidcap(vid_paths):
    for vid_path in vid_paths:
        vidcap = cv2.VideoCapture(str(vid_path))
        yield vidcap, vid_path
        vidcap.release()

def iterator_over_frames(vidcap, skip=2):
    duration = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
    height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width =  int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
    l = int(duration//skip)
    frame = np.zeros((height,width,3), dtype=np.uint8)
    for frame_pos in range(0, l*skip, skip):
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)
        res, frame[:] = vidcap.read()
        yield frame        
        
def snippet_is_ready(i, l): return i==l-1
def batch_is_ready(j, bs): return j==bs-1
        
def get_and_save_embs(b, model, vid_path, n_sn):
    vid_path = Path(vid_path)
    embs = model(b.permute(0,2,1,3,4).float())
    path = vid_path.parent/vid_path.stem
    path.mkdir(parents=True, exist_ok=True)
    for emb in embs:
        torch.save(emb, path/f'{n_sn:03d}.pth') ### REMEMBER TO PUT PATH!!

def create_embeddings(vid_paths, model, pl, bs, size, l, skip):
    fr_ts, snip, b = initialize_tensors(bs, l, size)
    for vidcap, vid_path in iterate_with_vidcap(vid_paths):   
        j = 0 # j contera' quanti snippets sono stati gia' messi nel batch       
        for n_fr, frame in enumerate(iterator_over_frames(vidcap, skip=skip)):
            i, n_sn = n_fr%l, n_fr//l  # i contera' il # di frames messo nello snippet, n_sn e' il numero di snips creati finora             
            fr_ts[:] = pl(frame)
            snip[i] = fr_ts            
            if snippet_is_ready(i,l): b[j] = snip; j += 1           
            if batch_is_ready(j,bs): get_and_save_embs(b, model, vid_path, n_sn); j = 0
            
        if not batch_is_ready(j,bs): get_and_save_embs(b[:j+1], model, vid_path, n_sn)  # if the video didnt fill the last batch, save nevertheless the 
                                                                                        # last created snipets

In [123]:
model = create_body(torchvision.models.video.r2plus1d_18, cut=-2).eval().float()

mean = [0.43216, 0.394666, 0.37645]
std = [0.22803, 0.22145, 0.216989]
bs, size, l, skip = 16, 224, 50, 2
pl = Pipeline([PILImage.create,
               Resize(size, method=ResizeMethod.Pad), 
               ToTensor(), 
               IntToFloatTensor(), 
               Normalize.from_stats(mean,std)])

path = Path(r'C:\Users\adria_000\Desktop\malas')
vid_paths = get_files(path, extensions='.AVI')[:1]

try: create_embeddings(vid_paths, model, pl, bs, size, l, skip)
except: vidcap.release(); raise Exception()

snip0 ready 



KeyboardInterrupt: 

In [99]:
for n_fr, frame in enumerate(iterator_over_frames(vidcap, skip=skip)):


torch.Size([16, 50, 3, 224, 224])

In [121]:
b.dtype

torch.float64